In [ ]:
import os
import pydicom
import traceback
import datetime
import numpy as np

In [1]:
def compute_suv_factor(image_path):

    estimated = False

    dcm_img = pydicom.dcmread(image_path, force=True)

    try:
        patient_weight = float(dcm_img.PatientWeight) * 1000
        if patient_weight <= 0:
            patient_weight = 75000
            estimated = True

    except:
        traceback.print_exc()
        patient_weight = 75000
        estimated = True
        
    try:
        scan_time = datetime.datetime.strptime(dcm_img.AcquisitionTime.split('.')[0], '%H%M%S')

        injection_time = datetime.datetime.strptime(dcm_img.RadiopharmaceuticalInformationSequence[0].RadiopharmaceuticalStartTime.split('.')[0],
                                                    '%H%M%S')

        # Half Life for Radionuclide # seconds
        half_life = float(dcm_img.RadiopharmaceuticalInformationSequence[0].RadionuclideHalfLife)
        # Total dose injected for Radionuclide
        injected_dose = float(dcm_img.RadiopharmaceuticalInformationSequence[0].RadionuclideTotalDose)

        # Calculate decay
        decay = np.exp(-np.log(2) * ((scan_time - injection_time).seconds) / half_life)
        # Calculate the dose decayed during procedure
        injected_dose_decay = injected_dose * decay

    except:
        traceback.print_exc()
        decay = np.exp(-np.log(2) * (1.75 * 3600) / 6588)  # 90 min waiting time, 15 min preparation
        injected_dose_decay = 420000000 * decay  # 420 MBq
        estimated = True

    suv_factor = patient_weight / injected_dose_decay
    return suv_factor, estimated





SyntaxError: unexpected EOF while parsing (<ipython-input-1-015c7f9d3305>, line 36)